In [25]:
import cv2
import mediapipe as mp
import numpy as np
import time

# Initialize MediaPipe utilities
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5)
mp_drawing = mp.solutions.drawing_utils

# Generic Angle Calculation Function
def calculate_angle(a, b, c):
    a = np.array(a); b = np.array(b); c = np.array(c)
    ba = a - b; bc = c - b
    dot_product = np.dot(ba, bc)
    magnitude_ba = np.linalg.norm(ba); magnitude_bc = np.linalg.norm(bc)
    if magnitude_ba == 0 or magnitude_bc == 0: return 0
    cosine_angle = np.clip(dot_product / (magnitude_ba * magnitude_bc), -1.0, 1.0)
    angle = np.arccos(cosine_angle)
    return np.degrees(angle)

# --- UPGRADED: The Exercise Engine with Posture Validation ---
def process_exercise(landmarks, config, state):
    if config.get('exercise_type', 'rep_based') == 'rep_based':
        # --- NEW: Posture Validation Step ---
        if 'posture_validation' in config:
            pv_config = config['posture_validation']
            p1 = [landmarks[pv_config['landmarks_to_track']['p1'].value].x, landmarks[pv_config['landmarks_to_track']['p1'].value].y]
            p2 = [landmarks[pv_config['landmarks_to_track']['p2'].value].x, landmarks[pv_config['landmarks_to_track']['p2'].value].y]
            p3 = [landmarks[pv_config['landmarks_to_track']['p3'].value].x, landmarks[pv_config['landmarks_to_track']['p3'].value].y]
            
            posture_angle = calculate_angle(p1, p2, p3)
            
            # Note the logic change here: angle should be LESS than the threshold for correct posture
            if posture_angle > pv_config['angle_threshold']:
                state['feedback'] = pv_config['feedback_map']['incorrect']
                state['color'] = (0, 0, 255) # Red for incorrect posture
                return state # Stop processing further

        # --- Standard Repetition Logic ---
        p1 = [landmarks[config['landmarks_to_track']['p1'].value].x, landmarks[config['landmarks_to_track']['p1'].value].y]
        p2 = [landmarks[config['landmarks_to_track']['p2'].value].x, landmarks[config['landmarks_to_track']['p2'].value].y]
        p3 = [landmarks[config['landmarks_to_track']['p3'].value].x, landmarks[config['landmarks_to_track']['p3'].value].y]
        p4 = [landmarks[config['landmarks_to_track']['p4'].value].x, landmarks[config['landmarks_to_track']['p4'].value].y]
        p5 = [landmarks[config['landmarks_to_track']['p5'].value].x, landmarks[config['landmarks_to_track']['p5'].value].y]
        p6 = [landmarks[config['landmarks_to_track']['p6'].value].x, landmarks[config['landmarks_to_track']['p6'].value].y]
        
        angle1 = calculate_angle(p1, p2, p3); angle2 = calculate_angle(p4, p5, p6)
        up_threshold = config['angle_thresholds']['up_threshold']
        down_threshold = config['angle_thresholds']['down_threshold']
        feedback_map = config['feedback_map']

        if not config.get('invert_stages'):
            if angle1 > up_threshold and angle2 > up_threshold:
                state['stage'] = "up"; state['feedback'] = feedback_map['up']; state['color'] = (245, 117, 66)
            elif angle1 < down_threshold or angle2 < down_threshold:
                if state['stage'] == 'up': state['counter'] += 1
                state['stage'] = "down"; state['feedback'] = feedback_map['down']; state['color'] = (0, 255, 0)
            else:
                if state['stage'] == 'up': state['feedback'] = feedback_map['transition_down']; state['color'] = (0, 0, 255)
                elif state['stage'] == 'down': state['feedback'] = feedback_map['transition_up']; state['color'] = (0, 0, 255)
        else:
            if angle1 < down_threshold and angle2 < down_threshold:
                state['stage'] = "down"; state['feedback'] = feedback_map['down']; state['color'] = (245, 117, 66)
            elif angle1 > up_threshold and angle2 > up_threshold:
                if state['stage'] == 'down': state['counter'] += 1
                state['stage'] = "up"; state['feedback'] = feedback_map['up']; state['color'] = (0, 255, 0)
            else:
                if state['stage'] == 'down': state['feedback'] = feedback_map['transition_up']; state['color'] = (0, 0, 255)
                elif state['stage'] == 'up': state['feedback'] = feedback_map['transition_down']; state['color'] = (0, 0, 255)

    elif config['exercise_type'] == 'timed':
        p1 = [landmarks[config['landmarks_to_track']['p1'].value].x, landmarks[config['landmarks_to_track']['p1'].value].y]
        p2 = [landmarks[config['landmarks_to_track']['p2'].value].x, landmarks[config['landmarks_to_track']['p2'].value].y]
        p3 = [landmarks[config['landmarks_to_track']['p3'].value].x, landmarks[config['landmarks_to_track']['p3'].value].y]
        angle = calculate_angle(p1, p2, p3)
        correct_form_threshold = config['angle_thresholds']['correct_form_threshold']
        if angle > correct_form_threshold:
            state['stage'] = 'correct'; state['feedback'] = config['feedback_map']['correct']; state['color'] = (0, 255, 0)
            if state['start_time'] is None: state['start_time'] = time.time()
            state['counter'] = time.time() - state['start_time']
        else:
            state['stage'] = 'incorrect'; state['feedback'] = config['feedback_map']['incorrect']; state['color'] = (0, 0, 255)
            state['start_time'] = None
            
    return state

# --- Main Application ---
cap = cv2.VideoCapture(0)

# --- Exercise Configurations ---
squat_config = {'name': 'SQUAT','landmarks_to_track': { 'p1': mp_pose.PoseLandmark.LEFT_HIP, 'p2': mp_pose.PoseLandmark.LEFT_KNEE, 'p3': mp_pose.PoseLandmark.LEFT_ANKLE, 'p4': mp_pose.PoseLandmark.RIGHT_HIP, 'p5': mp_pose.PoseLandmark.RIGHT_KNEE, 'p6': mp_pose.PoseLandmark.RIGHT_ANKLE },'angle_thresholds': { 'up_threshold': 160, 'down_threshold': 120 },'feedback_map': { 'up': 'Ready', 'down': 'Good Rep', 'transition_up': 'Get Up Straight', 'transition_down': 'Go Deeper' }}
bicep_curl_config = {'name': 'BICEP CURL','landmarks_to_track': { 'p1': mp_pose.PoseLandmark.LEFT_SHOULDER, 'p2': mp_pose.PoseLandmark.LEFT_ELBOW, 'p3': mp_pose.PoseLandmark.LEFT_WRIST, 'p4': mp_pose.PoseLandmark.RIGHT_SHOULDER, 'p5': mp_pose.PoseLandmark.RIGHT_ELBOW, 'p6': mp_pose.PoseLandmark.RIGHT_WRIST },'angle_thresholds': { 'up_threshold': 160, 'down_threshold': 40 },'feedback_map': { 'up': 'Ready', 'down': 'Good Rep', 'transition_up': 'Go Down Straight', 'transition_down': 'Curl Up' }}
push_up_config = {'name': 'PUSH-UP', 'invert_stages': True, 'landmarks_to_track': { 'p1': mp_pose.PoseLandmark.LEFT_SHOULDER, 'p2': mp_pose.PoseLandmark.LEFT_ELBOW, 'p3': mp_pose.PoseLandmark.LEFT_WRIST, 'p4': mp_pose.PoseLandmark.RIGHT_SHOULDER, 'p5': mp_pose.PoseLandmark.RIGHT_ELBOW, 'p6': mp_pose.PoseLandmark.RIGHT_WRIST },'angle_thresholds': { 'up_threshold': 160, 'down_threshold': 90 },'feedback_map': { 'up': 'Good Rep', 'down': 'Ready', 'transition_up': 'Push Up', 'transition_down': 'Go Lower' }}
lunge_config = {'name': 'LUNGE','landmarks_to_track': { 'p1': mp_pose.PoseLandmark.LEFT_HIP, 'p2': mp_pose.PoseLandmark.LEFT_KNEE, 'p3': mp_pose.PoseLandmark.LEFT_ANKLE, 'p4': mp_pose.PoseLandmark.RIGHT_HIP, 'p5': mp_pose.PoseLandmark.RIGHT_KNEE, 'p6': mp_pose.PoseLandmark.RIGHT_ANKLE },'angle_thresholds': { 'up_threshold': 160, 'down_threshold': 100 },'feedback_map': { 'up': 'Ready', 'down': 'Good Rep', 'transition_up': 'Push Back Up', 'transition_down': 'Step Forward' }}
plank_config = {'name': 'PLANK', 'exercise_type': 'timed','landmarks_to_track': { 'p1': mp_pose.PoseLandmark.LEFT_SHOULDER, 'p2': mp_pose.PoseLandmark.LEFT_HIP, 'p3': mp_pose.PoseLandmark.LEFT_ANKLE, 'p4': mp_pose.PoseLandmark.RIGHT_SHOULDER, 'p5': mp_pose.PoseLandmark.RIGHT_HIP, 'p6': mp_pose.PoseLandmark.RIGHT_ANKLE },'angle_thresholds': { 'correct_form_threshold': 160 },'feedback_map': { 'correct': 'Hold Position', 'incorrect': 'Straighten Back', 'not_ready': 'Get into Position' }}
overhead_press_config = {'name': 'OVERHEAD PRESS','invert_stages': True,'landmarks_to_track': { 'p1': mp_pose.PoseLandmark.LEFT_SHOULDER, 'p2': mp_pose.PoseLandmark.LEFT_ELBOW, 'p3': mp_pose.PoseLandmark.LEFT_WRIST, 'p4': mp_pose.PoseLandmark.RIGHT_SHOULDER, 'p5': mp_pose.PoseLandmark.RIGHT_ELBOW, 'p6': mp_pose.PoseLandmark.RIGHT_WRIST },'angle_thresholds': { 'up_threshold': 160, 'down_threshold': 90 },'feedback_map': { 'up': 'Good Rep', 'down': 'Ready', 'transition_up': 'Press Up', 'transition_down': 'Lower Slowly' }}
jumping_jacks_config = {'name': 'JUMPING JACKS','invert_stages': True,'landmarks_to_track': { 'p1': mp_pose.PoseLandmark.LEFT_HIP, 'p2': mp_pose.PoseLandmark.LEFT_SHOULDER, 'p3': mp_pose.PoseLandmark.LEFT_ELBOW, 'p4': mp_pose.PoseLandmark.RIGHT_HIP, 'p5': mp_pose.PoseLandmark.RIGHT_SHOULDER, 'p6': mp_pose.PoseLandmark.RIGHT_ELBOW },'angle_thresholds': { 'up_threshold': 140, 'down_threshold': 60 },'feedback_map': { 'up': 'Good Rep', 'down': 'Ready', 'transition_up': 'Arms Up!', 'transition_down': 'Arms Down!' }}
glute_bridge_config = {'name': 'GLUTE BRIDGE','invert_stages': True,'landmarks_to_track': { 'p1': mp_pose.PoseLandmark.LEFT_SHOULDER, 'p2': mp_pose.PoseLandmark.LEFT_HIP, 'p3': mp_pose.PoseLandmark.LEFT_KNEE, 'p4': mp_pose.PoseLandmark.RIGHT_SHOULDER, 'p5': mp_pose.PoseLandmark.RIGHT_HIP, 'p6': mp_pose.PoseLandmark.RIGHT_KNEE },'angle_thresholds': { 'up_threshold': 160, 'down_threshold': 100 },'feedback_map': { 'up': 'Good Squeeze', 'down': 'Ready', 'transition_up': 'Lift Hips', 'transition_down': 'Lower Slowly' }}
high_knees_config = {'name': 'HIGH KNEES','landmarks_to_track': { 'p1': mp_pose.PoseLandmark.LEFT_SHOULDER, 'p2': mp_pose.PoseLandmark.LEFT_HIP, 'p3': mp_pose.PoseLandmark.LEFT_KNEE, 'p4': mp_pose.PoseLandmark.RIGHT_SHOULDER, 'p5': mp_pose.PoseLandmark.RIGHT_HIP, 'p6': mp_pose.PoseLandmark.RIGHT_KNEE },'angle_thresholds': { 'up_threshold': 150, 'down_threshold': 100 },'feedback_map': { 'up': 'Ready', 'down': 'Good Rep', 'transition_up': 'Knee Down', 'transition_down': 'Knee Up!' }}
burpees_config = {'name': 'BURPEES','landmarks_to_track': { 'p1': mp_pose.PoseLandmark.LEFT_SHOULDER, 'p2': mp_pose.PoseLandmark.LEFT_HIP, 'p3': mp_pose.PoseLandmark.LEFT_KNEE, 'p4': mp_pose.PoseLandmark.RIGHT_SHOULDER, 'p5': mp_pose.PoseLandmark.RIGHT_HIP, 'p6': mp_pose.PoseLandmark.RIGHT_KNEE },'angle_thresholds': { 'up_threshold': 160, 'down_threshold': 100 },'feedback_map': { 'up': 'Ready', 'down': 'Good Rep', 'transition_up': 'Jump Up', 'transition_down': 'Chest to Floor' }}
bent_over_row_config = {
    'name': 'BENT OVER ROW',
    'landmarks_to_track': { 'p1': mp_pose.PoseLandmark.LEFT_SHOULDER, 'p2': mp_pose.PoseLandmark.LEFT_ELBOW, 'p3': mp_pose.PoseLandmark.LEFT_WRIST, 'p4': mp_pose.PoseLandmark.RIGHT_SHOULDER, 'p5': mp_pose.PoseLandmark.RIGHT_ELBOW, 'p6': mp_pose.PoseLandmark.RIGHT_WRIST },
    'angle_thresholds': { 'up_threshold': 160, 'down_threshold': 90 },
    'feedback_map': { 'up': 'Ready', 'down': 'Good Squeeze', 'transition_up': 'Lower Slowly', 'transition_down': 'Pull!' },
    'posture_validation': {
        'landmarks_to_track': { 'p1': mp_pose.PoseLandmark.LEFT_SHOULDER, 'p2': mp_pose.PoseLandmark.LEFT_HIP, 'p3': mp_pose.PoseLandmark.LEFT_KNEE },
        'angle_threshold': 110, # User's hip angle must be LESS than this
        'feedback_map': { 'incorrect': 'Bend Over More' }
    }
}

# --- State Management ---
current_exercise_config = squat_config
state = {'counter': 0, 'stage': 'up', 'feedback': 'Ready', 'color': (245, 117, 66), 'start_time': None}

while cap.isOpened():
    success, frame = cap.read()
    if not success: continue

    image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose.process(image_rgb)
    
    if results.pose_landmarks:
        state = process_exercise(results.pose_landmarks.landmark, current_exercise_config, state)
        mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS, mp_drawing.DrawingSpec(color=state['color'], thickness=2, circle_radius=2), mp_drawing.DrawingSpec(color=state['color'], thickness=2, circle_radius=2))
    else:
        state['feedback'] = 'No Pose Detected'
        
    # --- VISUALIZATION ---
    frame = cv2.flip(frame, 1)
    cv2.rectangle(frame, (15, 15), (450, 100), (0,0,0), -1)
    cv2.putText(frame, 'EXERCISE', (25, 40), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255,255,255), 2, cv2.LINE_AA)
    cv2.putText(frame, current_exercise_config['name'], (30, 85), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (255,255,255), 2, cv2.LINE_AA)
    display_counter = state['counter']; counter_label = 'REPS'
    if current_exercise_config.get('exercise_type') == 'timed': counter_label = 'TIME (s)'; display_counter = int(display_counter)
    cv2.putText(frame, counter_label, (200, 40), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255,255,255), 2, cv2.LINE_AA)
    cv2.putText(frame, str(display_counter), (205, 85), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255,255,255), 2, cv2.LINE_AA)
    cv2.putText(frame, 'FEEDBACK', (310, 40), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255,255,255), 2, cv2.LINE_AA)
    cv2.putText(frame, state['feedback'], (315, 85), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255,255,255), 2, cv2.LINE_AA)
    
    # Key press logic
    key = cv2.waitKey(5) & 0xFF
    def reset_state(config):
        is_press = config.get('invert_stages', False)
        stage = 'down' if is_press else 'up'
        return {'counter': 0, 'stage': stage, 'feedback': 'Ready', 'color': (245, 117, 66), 'start_time': None}

    if key == ord('s'): current_exercise_config = squat_config; state = reset_state(squat_config)
    elif key == ord('b'): current_exercise_config = bicep_curl_config; state = reset_state(bicep_curl_config)
    elif key == ord('p'): current_exercise_config = push_up_config; state = reset_state(push_up_config)
    elif key == ord('l'): current_exercise_config = lunge_config; state = reset_state(lunge_config)
    elif key == ord('k'): current_exercise_config = plank_config; state = {'counter': 0, 'stage': 'start', 'feedback': 'Get Ready', 'color': (245, 117, 66), 'start_time': None}
    elif key == ord('o'): current_exercise_config = overhead_press_config; state = reset_state(overhead_press_config)
    elif key == ord('j'): current_exercise_config = jumping_jacks_config; state = reset_state(jumping_jacks_config)
    elif key == ord('g'): current_exercise_config = glute_bridge_config; state = reset_state(glute_bridge_config)
    elif key == ord('h'): current_exercise_config = high_knees_config; state = reset_state(high_knees_config)
    elif key == ord('u'): current_exercise_config = burpees_config; state = reset_state(burpees_config)
    elif key == ord('r'): current_exercise_config = bent_over_row_config; state = reset_state(bent_over_row_config)
    elif key == ord('q'): break
    
    cv2.imshow('AI Fitness Coach', frame)

cap.release()
cv2.destroyAllWindows()



KeyboardInterrupt: 